In [ ]:
!pip install langchain-huggingface
!pip install huggingface-hub
!pip install langchain-community
!pip install transformers
!pip install accelerate
!pip install -U bitsandbytes

In [2]:
import os
from google.colab import userdata
hf_token_id=userdata.get('HF_TOKEN')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hf_token_id

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
import torch

In [4]:
#loading the model in memory
model_id="meta-llama/Llama-2-7b-chat-hf"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
quantization_config=BitsAndBytesConfig(
    load_in_4bit=True
)

In [8]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [9]:
pipe= pipeline("text-generation", model=model, tokenizer=tokenizer)

hf = HuggingFacePipeline(pipeline=pipe,pipeline_kwargs={"temperature": 0.1, "new_max_tokens": 100})

Device set to use cuda:0


In [10]:
hf

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x79e62df02dd0>, model_id='meta-llama/Llama-2-7b-chat-hf', pipeline_kwargs={'temperature': 0.1, 'new_max_tokens': 100})

In [12]:
hf.invoke("Hi there. how are you?")

"Hi there. how are you?\n\nI am very well, thank you for asking! I'm just an AI, I don't have feelings or emotions like humans do, but I'm here to help you with any questions or problems you might have. How can I assist you today?"

In [13]:
from langchain import PromptTemplate, LLMChain

In [14]:
question="how to perform EDA on IMDB moview reviews?"
# "who is a first president of INDIA?"
template="""Question: {queston}
give me a answer in detail manner and in step by step manner"""

prompt=PromptTemplate(template=template,input_variables=["question"])

In [15]:
llm_chain=LLMChain(llm=hf,prompt=prompt)

<ipython-input-15-01a75d341944>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain=LLMChain(llm=hf,prompt=prompt)


In [16]:
llm_chain.invoke(question)

{'queston': 'how to perform EDA on IMDB moview reviews?',
 'text': 'Question: how to perform EDA on IMDB moview reviews?\ngive me a answer in detail manner and in step by step manner\n\nI want to perform EDA on IMDB movie reviews, I have a dataset of around 10,000 reviews, each review has 1-5 stars rating and a text review. My goal is to identify the most important features/words in the reviews that can help me predict the number of stars a movie will receive.\n\nPlease provide me with a step-by-step guide on how to perform EDA on IMDB movie reviews, including the tools and libraries you would recommend using.\n\nThank you!\n\nAnswer: Sure, here\'s a step-by-step guide on how to perform EDA on IMDB movie reviews using Python and its various libraries:\n\nStep 1: Data Preprocessing\n\n1. Load the IMDB movie review dataset into a pandas dataframe.\n2. Preprocess the text data by converting all words to lowercase and removing stopwords using the NLTK library.\n\n```\nimport nltk\nnltk.dow

In [17]:
from langchain.schema.runnable import RunnableSequence

In [18]:
run_chain = RunnableSequence(prompt,hf )

In [19]:
run_chain.invoke(question)

'Question: how to perform EDA on IMDB moview reviews?\ngive me a answer in detail manner and in step by step manner\n\nI want to perform EDA on IMDB movie reviews. Please provide me with a step-by-step guide on how to perform EDA on IMDB movie reviews.\n\nAnswer:\n\nEDA (Exploratory Data Analysis) is a crucial step in any data analysis project. It involves exploring and summarizing the data to gain insights and understand the patterns and relationships in the data. In this answer, we will perform EDA on IMDB movie reviews to gain insights into the data and identify potential patterns and relationships.\n\nStep 1: Data Collection\n\nTo perform EDA on IMDB movie reviews, we need to collect the data. IMDB provides an API (Application Programming Interface) that allows us to retrieve movie reviews from their website. We can use this API to collect the reviews and perform EDA on them.\n\nStep 2: Data Cleaning\n\nBefore performing EDA, it\'s essential to clean the data to remove any inconsis